# Actividad M1
José Sebastián Naranjo Zamudio A01066941



## Imports

Antes de empezar a crear el modelo del juego de la vida con multiagentes es necesario tener instalado los siguientes paquetes:
- `python`: asegúrense de usar la versión 3+.
- `mesa`: el framework de Python para el modelado de agentes.
- `numpy`: es una biblioteca de Python para el manejo de matrices, arreglos, manipulación matemática, lógica y mucho más.
- `matplotlib`: es una biblioteca para crear visualizaciones estáticas, animadas e interactivas en Python.

Para poder modelar el juego de la vida usando el framework de `mesa` es necesario importar dos clases: una para el modelo general, y otro para los agentes. 

In [78]:
!pip3 install mesa

In [79]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `SingleGrid` que fuerza un solo objeto por celda.
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

## Crear el modelo

Antes que nada el presente modelo se encuentra basado en el [tutorial introductorio](https://mesa.readthedocs.io/en/master/tutorials/intro_tutorial.html). Lo modifiqué un poco para que funcionara para el presente problema pero en esencia es lo mismo.

In [80]:
class RandomWalker(Agent):
    """
    Class implementing random walker methods in a generalized manner.
    Not indended to be used on its own, but to inherit its methods to multiple
    other agents.
    """

    grid = None
    x = None
    y = None
    moore = True

    def __init__(self, unique_id, pos, model, moore=True):
        """
        grid: The MultiGrid object in which the agent lives.
        x: The agent's current x coordinate
        y: The agent's current y coordinate
        moore: If True, may move in all 8 directions.
                Otherwise, only up, down, left, right.
        """
        super().__init__(unique_id, model)
        self.pos = pos
        self.moore = moore

    def random_move(self):
        """
        Step one cell in any allowable direction.
        """
        # Pick the next cell from the adjacent cells.
        next_moves = self.model.grid.get_neighborhood(self.pos, self.moore, True)
        next_move = self.random.choice(next_moves)
        # Now move:
        self.model.grid.move_agent(self, next_move)

In [86]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        grid[x][y] = cell_content.live
    return grid

class Dirty_Tile(Agent):
    def __init__(self, unique_id,model,pos):
        super().__init__(unique_id, model,pos)
        self.pos = pos
    
        
class Vacuum_Robot(RandomWalker):
    def __init__(self, unique_id, model,pos):
        super().__init__(unique_id, model,pos)
        
    def step(self):
        self.random_move()
        
        x, y = self.pos
        this_cell = self.model.grid.get_cell_list_contents([self.pos])
        tile = [obj for obj in this_cell if isinstance(obj, Dirty_Tile)]
        
        if len(tile) > 0:
            tile_to_clean = self.random.choice(tile)

            # Limpiar el tile
            self.model.grid._remove_agent(self.pos, tile_to_clean)
            self.model.schedule.remove(tile_to_clean)
            
class GameLifeModel(Model):
    
    def __init__(self,T,V, width, height):
        self.num_agents_tiles = T
        self.num_agents_vacuums=V
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        
      # Create tiles
        for i in range(self.num_agents_tiles):
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            a = Dirty_Tile(i,self(x,y))
            self.grid.place_agent(a, (x, y))
            
       # Create vacuums
        for i in range(self.num_agents_vacuums):
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            b= Vacuum_Robot(i,self(x,y))
            self.grid.place_agent(b,(x,y))

    
        # Aquí definimos con colector para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
        
            
    def step(self):
        '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
        '''
        self.datacollector.collect(self)
        self.schedule.step()

A continuación corremos el modelo

In [87]:
# Definimos el tamaño del Grid
width=10
height=10

# Definimos el número de agentes
Tiles = 3
Vacuum = 3

#Steps
steps=10

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
model = GameLifeModel(Tiles,Vacuum,width, height)
for i in range(time):
    model.step()

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

TypeError: 'GameLifeModel' object is not callable

Obtenemos la información que almacenó el colector, este nos entregará un DataFrame de pandas que contiene toda la información.

In [83]:
all_grid = model.datacollector.get_model_vars_dataframe()

Graficamos la información usando `matplotlib`

In [84]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='YlGnBu')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

IndexError: single positional indexer is out-of-bounds

In [85]:
anim

NameError: name 'anim' is not defined

## Conclusiones



In [88]:
'''Es un tanto aparente que el número de agentes aspiradora puede ayudar a mejorar el tiempo requerido 
para limpiar la superficie, ya que si se posicionará un agente por tile existente la limpieza seria instantea
y sin suciedad restante. 
Sin embargo, en aplicaciones del mundo real existen limitaciones de recursos y espacio por lo que lo anterior no es un opción, 
lo mas apto seria dotar de mayores capacidades a pocos agentes (por ejemplo la habilidad de recordar dónde estuvo)
asi mejorando la limpieza y el timepo que esta actividad conlleva.'''

SyntaxError: invalid syntax (<ipython-input-88-a430853c56c8>, line 1)